In [1]:
import open3d as o3d
import copy
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import time

# 1. Getting started with Open3D
## 1.1 Converting RGBD image into Point Cloud


In your robotics journey, it is common to be given just the depth images along with camera parameters in a generic dataset and you'd want to build a 3D data representation out of it, for example, a point cloud. You will understand the math behind these concepts in detail during Vision classes, for now, you can use the in-built functions as a black box.
- Below are the given RGB and D images from SUN RGB-D dataset ([S.Song, CVPR 2015](https://rgbd.cs.princeton.edu/)). 
![low-res-RGBD.resized.png](./misc/low-res-RGBD.png)

- Read these two images `color.jpg` and `depth.png` given in current folder using Open3D. Convert it to a point cloud using the default camera parameters (`o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault`).
Then,
- Create a "world" frame $A$ and combine this (just use $+$ operator) with the above point cloud and save it as `scene.pcd`. Put it aside for now.
- Write a simple function `one_one` to visualize `scene.pcd`.

In [2]:
color_raw = o3d.io.read_image("color.jpg")
depth_raw = o3d.io.read_image("depth.png")
rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format(color_raw, depth_raw)
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame( size=1, origin=[1, 1, 1])
world_frame.paint_uniform_color([0, 0, 0])

frame = o3d.geometry.TriangleMesh.sample_points_uniformly(world_frame, number_of_points=15000, use_triangle_normal=False, seed=- 1)
o3d.io.write_point_cloud("scene.pcd", pcd+frame)


True

Displaying `scene.pcd`.

In [3]:
#uncomment the following and add input parameters if any
def one_one(file):
    pcd = o3d.io.read_point_cloud(file)
    o3d.visualization.draw_geometries([pcd])
one_one("scene.pcd")

# 2) Rotations, Euler angles and Gimbal Lock

## Rotating an object

The objective here is to roughly simulate an object moving on a ground.

- Generate a cube at some point on the ground and create another frame $B$ at the center of this object. Combine these both as a single point cloud `cube.pcd`. (You can pick a point on the ground by using the `get_picked_points` method of the class `open3d.visualization.VisualizerWithEditing`.)
- Now read both the point clouds `scene.pcd` and `cube.pcd` in a script. Whatever tasks you do below are on the object `cube.pcd` (along with the axes $B$) with `scene.pcd` in the background (static).
- Given a sequence of **ZYX Euler** angles $[30^{\circ}, 90^{\circ}, 45^{\circ}]$, generate the rotation. In our case, our object (with its respective axis) undergoes rotation with the background being fixed (with its respective axis).
- Note: we will be using the standard **ZYX** Euler angle convention.
- Write a function `two_one` to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*


In [4]:
def pick_points(pcd):
    print("")
    print("Pick point using [shift + left click]")
    print("Afther picking, press q for close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  
    vis.destroy_window()
    print("")
    return vis.get_picked_points()


pcd = o3d.io.read_point_cloud("scene.pcd")
picked = pick_points(pcd)
print(picked)

picked_coord = pcd.points[picked[0]]
print(picked_coord)
mesh_box = o3d.geometry.TriangleMesh.create_box(width=1, height=1,depth=1)
mesh_box.compute_vertex_normals()
mesh_box.paint_uniform_color([0.9, 0.1, 0.1])
box = o3d.geometry.TriangleMesh.sample_points_uniformly(mesh_box, number_of_points=100500, use_triangle_normal=False, seed=- 1)
center = o3d.geometry.PointCloud.get_center(box)
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame( size=1.2, origin=[-0.5, -0.5, -0.5])
frame = o3d.geometry.TriangleMesh.sample_points_uniformly(mesh_frame, number_of_points=15000, use_triangle_normal=False, seed=- 1)
pcd = frame+ box
# o3d.visualization.draw_geometries([pcd])
o3d.io.write_point_cloud("cube.pcd", pcd)

def two_one():
    def draw(cube,scene,picked_coord):
        vis = o3d.visualization.Visualizer()
        vis.create_window()
        vis.add_geometry(cube)
        vis.add_geometry(scene)
        cx = picked_coord[0] + 0.5
        cy = picked_coord[1] + 0.5
        cz = picked_coord[2] + 0.5
        k = 200
        for i in range(k):
            time.sleep(0.0065)
            R = cube.get_rotation_matrix_from_zyx(((np.pi/(6*k)),0,0))
            cube.rotate(R, (cx,cy,cz))
            vis.update_geometry(cube)
            vis.poll_events()
            vis.update_renderer()
        Rz = cube.get_rotation_matrix_from_zyx((np.pi/6,0,0))
        for i in range(k):
            time.sleep(0.0065)
            R = cube.get_rotation_matrix_from_zyx((0,np.pi/(2*k),0))
            cube.rotate((Rz@R)@Rz.T,(cx,cy,cz))
            vis.update_geometry(cube)
            vis.poll_events()
            vis.update_renderer()
        Ry = cube.get_rotation_matrix_from_zyx((0,np.pi/2,0))
        for i in range(k):
            time.sleep(0.0065)
            R = cube.get_rotation_matrix_from_zyx((0,0,np.pi/(4*k)))
            cube.rotate((Ry@R)@Ry.T,(cx,cy,cz))
            vis.update_geometry(cube)
            vis.poll_events()
            vis.update_renderer()
        vis.destroy_window()
        return True
    cube = o3d.io.read_point_cloud("cube.pcd")
    cube.translate(picked_coord)                                          
    scene = o3d.io.read_point_cloud("scene.pcd")
    draw(cube,scene,picked_coord)  



Pick point using [shift + left click]
Afther picking, press q for close the window
[Open3D INFO] Picked point #185572 (-0.69, -0.94, -3.4) to add in queue.

[185572]
[-0.68986952 -0.93835336 -3.43300009]




Calling the function `two_one` here showing the animation described above.

In [5]:

two_one()

## 2.2 Euler angle & Gimbal lock

Code the following yourself from scratch (Refer Craig book - Section: $Z-Y-X$ Euler angles - same conventions/notations followed).

- Case 1: Given the rotation matrix $M_{given}$ below, extract Euler angles $\alpha , \beta ,\gamma$. Convert it back to the rotation matrix $M_{recovered}$ from Euler angles.

    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] $$

    After coding it from scratch, check your calculations using `scipy.spatial.transform.Rotation`. (Mandatory)

- Case 2: Given the rotation matrix $N_{given}$, extract Euler angles, and convert back $N_{recovered}$.

    $$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] $$

    Again use `scipy` and check its output. If `scipy` is showing any warnings on any of the above cases, explain it in "**Questions for 2.2**" (last question). Write code in the next cell.
    
- (Optional) Case 3: Do the above two for quaternion using scipy functions, i.e. given the rotation matrix, extract quaternion and convert back.

In [6]:
M_given =  np.array([[0.26200263, -0.19674724, 0.944799],
                     [0.21984631, 0.96542533, 0.14007684],
                     [-0.93969262, 0.17101007, 0.29619813]])

N_given = np.array([[0,-0.173648178,0.984807753],
                    [0, 0.984807753, 0.173648178],
                    [-1, 0, 0]])

In [7]:
from scipy.spatial.transform import Rotation as R

def getEuler(M_given):
    alpha = 0
    beta = 0
    yamma = 0
    if(M_given[2][0]!=1 and M_given[2][0]!=-1 ):
        beta = np.arctan2(-1*M_given[2][0],np.sqrt(pow(M_given[0][0],2)+  pow(M_given[1][0],2))) 
        alpha = np.arctan2(M_given[1][0]/np.cos(beta) , M_given[0][0]/np.cos(beta))
        gamma = np.arctan2(M_given[2][1]/np.cos(beta) , M_given[2][2]/np.cos(beta))
    else:
        if(M_given[2][0]==-1):
            beta = np.pi/2
            gamma = alpha + np.arctan2(M_given[0][1],M_given[0][2])
        else:
            beta = -np.pi/2
            gamma = -alpha + np.arctan2(-M_given[0][1],-M_given[0][2])

    return convert_deg(alpha,beta,gamma)

def convert_deg(alpha,beta,gamma):
    return alpha*180/np.pi,beta*180/np.pi,gamma*180/np.pi

def convert_rad(alpha,beta,gamma):
    return alpha*np.pi/180,beta*np.pi/180,gamma*np.pi/180

def reconstructMat(alpha,beta,gamma):
   
    Rz = np.array([[np.cos(alpha),-1*np.sin(alpha),0],
                   [np.sin(alpha),np.cos(alpha),0],
                   [0,0,1]])
    Ry = np.array([[np.cos(beta),0,np.sin(beta)],
                   [0,1,0],
                   [-1*np.sin(beta),0,np.cos(beta)]])
    Rx = np.array([[1,0,0],
                   [0,np.cos(gamma),-1*np.sin(gamma)],
                   [0,np.sin(gamma),np.cos(gamma)]])
    R = np.matmul(np.matmul(Rz,Ry),Rx)
    return R

def check(M_given):
    alpha,beta,gamma = getEuler(M_given)

    alpha,beta,gamma= convert_rad(alpha,beta,gamma)
    mat = reconstructMat(alpha,beta,gamma)
    alpha,beta,gamma = convert_deg(alpha,beta,gamma)
    print("\n Reconstructed angles from code written from scratch α,β,γ = ", alpha," ",beta," ",gamma)
    print("\n Reconstructed Matrix from code written from scratch")
    print(mat)
    M = R.from_matrix(M_given)
    # checking
    r = M.from_euler('ZYX', [alpha,beta,gamma], degrees = True)
    print("Reconstructed angles as per the Scipy's functions [α,β,γ] = ",M.as_euler('ZYX', degrees=True))
    print("\n Reconstructed Matrix from Scipy's functions ")
    print(r.as_matrix())
    return [alpha,beta,gamma],mat

print("* Case 1 *")

my_array_case1,M_recovered = check(M_given)
print("\n * Case 2*")

my_array_case2,N_recovered = check(N_given)


##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

* Case 1 *

 Reconstructed angles from code written from scratch α,β,γ =  39.99999997427293   70.00000000765922   29.99999998709331

 Reconstructed Matrix from code written from scratch
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]
Reconstructed angles as per the Scipy's functions [α,β,γ] =  [40.00000028 70.00000021 30.00000046]

 Reconstructed Matrix from Scipy's functions 
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]

 * Case 2*

 Reconstructed angles from code written from scratch α,β,γ =  0.0   90.0   -10.000000018915026

 Reconstructed Matrix from code written from scratch
[[ 6.12323400e-17 -1.73648178e-01  9.84807753e-01]
 [ 0.00000000e+00  9.84807753e-01  1.73648178e-01]
 [-1.00000000e+00 -1.06328843e-17  6.03020831e-17]]
Reconstructed angles as per the Scipy's functions [α,β,γ] =  [10.00000002 90.          0.        ]

 Reconstructed M

/home/lenovo/.local/lib/python3.7/site-packages/scipy/spatial/transform/rotation.py:135: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  warnings.warn("Gimbal lock detected. Setting third angle to zero since"


We use np.arctan2 fucntion since it returns array of angles in radians, in the range [-pi, pi] and we required the range to be -pi to pi in euler angles. On the other hand np.arctan would return in range [-pi/2, pi/2] which was not what we required.

In [8]:
# Uncomment and replace my_array_case1 with your array.
print("My Euler angles for case 1 are" + str(my_array_case1))

My Euler angles for case 1 are[39.99999997427293, 70.00000000765922, 29.99999998709331]


  - Were you able to recover back your rotation matrix when you converted it from Euler angles? Why/why not? Replace `M_given` and `M_recovered` with your matrices below and explain "why/why not" after this code snippet.

In [9]:
# Uncomment and Replace M_given and M_recovered with your matrices below.
error = np.linalg.norm(logm(M_given @ M_recovered.T))
print("For case 1, it is " + str(error<0.0001) + " I could recover the original matrix.")

For case 1, it is True I could recover the original matrix.


 *As we see, we are accurately able to recover the rotation matrix by coding from scratch as well as using scipy for case 1*

### Repeating the above for Case 2.

In [10]:
print("My Euler angles for case 2 are" + str(my_array_case2))
error = np.linalg.norm(logm(N_given @ N_recovered.T))
print("For case 2, it is " + str(error<0.0001) + " I could recover the original matrix.")

My Euler angles for case 2 are[0.0, 90.0, -10.000000018915026]
For case 2, it is True I could recover the original matrix.


* Is there any problem with Euler angle representation for Case 2? 

    * Ans: Yes, we did face a problem of Gimbal lock in case 2. Since  cosβ = 0 in case 2, i.e R31 element of rotation matrix is -1 which corresponds to β=pi/2. <br>
    When we try to solve for values of alpha and gamma we get a division by 0 term and hence needed to explicityly handle the case of gimbal lock.  <br>
    Gimbal lock occurs when cosβ = 0  i.e β = pi/2 or -pi/2(R31 = 1 or -1) <br>
   Here a unique solution set of Euler Angles doesnt exist. <br>
  Hence in such cases, we check if R31 is 1 or -1 and find beta accordingly.  Then make any one anlge among alpha or gamma to be zero and caclualte the other. We calculated using alpha = 0 as shown above in the code.
  <br>
    
* What is this Gimbal Lock?
    * Ans: *When the angle β becomes + or - 90.0◦ Gimbal lock situtation arises. <br>It is situation when a degree of freedom is lost when an axis alignes with another (The two axis are parralel to each other)*

- When you used `scipy.spatial.transform.Rotation` for the above 2 cases,
    - Have you used `zyx` above in `r.as_euler('')` argument? Why or why not? Explain the difference between extrinsic and instrinsic rotations with equivalent technical names from Craig book? <br>
        Ans: We used ZYX which is for intrisic euler angle rotations in scipy instead of using zyx which corresponds to extrinsic angles. We needed extrinsic rotations here. <br>
        * Extrinsic means the original coordinate system remains motionless as the rotations go. <br>
        * Intrinsic means the rotation is always based on the rotating coordinate system which is solidary with the moving object. <br>
        Since it was asked that the original coordinate system, i.e scene should remain fixed and olny cube must rotate, we needed extrinsic rotations.

    - Has `scipy` shown any warnings on any of the above cases? If yes, explain it.
        * Ans: *Yes, we get a warning for Case2, where there was a Gimbal lock situation. This occurs when there is one less degree of freedom for rotation, which results in infinite possible solutions given the order of rotation of axes remains unchanged.*
    - (Optional) For Case 3 above (quaternion) which you did using scipy, did you observe any problem with quaternion? Depending on your observations, which is better? Quaternion or Euler angles? And why?
         * Ans: Quaternions have 4 gimbals hence they dont have  problems of gimbal lock. <br>
         It is better to use quaternions instead of the Euler angles as 3 degrees of freedom are always ensured in the Quaternion and additionally it also prevents gimbal lock. <br>


## 2.3 Rotation matrix as an Operator
This question will help you in your understanding of [Rotator-transform (Vector-frame) equivalence](https://www.notion.so/saishubodh/Lecture-2-Transformations-11d69d8cef2d4cd195a98fa7d33224e1#f90ece4f5e374743bfed47e46a83ecfe).

![image.png](./misc/xyz-frame.png)
Consider the frame $XYZ$ in the above image. Say you have a vector $x_1=[0,\sqrt{3},0]$. Now you want to rotate it such that you end up at $x_2=[1,1,1]$ through a sequence of Euler angle rotations. Your goal is to find out those $\alpha, \beta \: \& \: \gamma$ ($ZYX$). We will follow this order whenever we refer to it below.

First, properly understand the so-called "Rotator-transform equivalence" to figure out what are the terms of rotation matrix. Then, put the math on paper and you will end up with a set of non-linear equations. Write the set of linear equations in LaTeX here:

$$ \textbf{The equations} $$
$$
\textit{X}^{'} = \textit{X} * ( \cos{\alpha} * \cos{\beta} ) +  \textit(Y) * (\cos{\alpha} * \sin{\beta} * \sin{\gamma} - \sin{\alpha} * \cos{\gamma}) + \textit{Z} * (\cos{\alpha} * \sin{\beta} * \cos{\gamma} + \sin{\alpha} * \sin{\gamma})
$$
$$
\textit{Y}^{'} = \textit{X} * ( \sin{\alpha} * \cos{\beta} ) +  \textit(Y) * (\sin{\alpha} * \sin{\beta} * \sin{\gamma} + \cos{\alpha} * \cos{\gamma}) + \textit{Z} * (\sin{\alpha} * \sin{\beta} * \cos{\gamma} - \cos{\alpha} * \sin{\gamma})
$$
$$
\textit{Z}^{'} = \textit{X} * ( -\sin{\beta} ) +  \textit(Y) * (\cos{\beta} * \sin{\gamma}) + \textit{Z} * (\cos{\beta} * \cos{\gamma} )
$$

On substituting the values of X,Y,Z to be $$(0,\sqrt{3},0)$$ we get:

$$ cos\alpha sin\beta sin\gamma - sin\alpha cos\gamma = 1/\sqrt3 $$ 
$$ sin\alpha sin\beta sin\gamma + cos\alpha cos\gamma = 1/\sqrt3 $$ 
$$ cos\beta sin\gamma = 1/\sqrt3 $$ 


Solve these equations using `fsolve` from `scipy.optimize` as follows: (Come back and answer the following questions after coding it in the next block)
- `case1`: First, solve it with an initialization of (0,0,0). Check if your answer is correct using `np.isclose`.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_1, b_1, c_1$ = -41.062913795413124, 5765.546112808945, 35.455164399949105* 
- `case2`: Now, forget about the solver for a moment: Can you visualize and think of sequence of rotations one by one to reach the final position (which is different than previous set of rotations)? Now, validate your answer by giving (your answer $\pm 5$) as initialization.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_2, b_2, c_2$ = 315.0(or -45 degree), 0.0, 35.26438968275467.*

In [ ]:
x_1 = np.array([0,np.sqrt(3),0])
x_2 = np.array([1,1,1])

In [12]:
import numpy as np
from scipy.optimize import fsolve

def solve(arr):
    x_1 = np.array([0,np.sqrt(3),0])
    x_2 = np.array([1,1,1])
    alpha,beta,gamma = arr[0],arr[1],arr[2]
    Rz = np.array([[np.cos(alpha),-1*np.sin(alpha),0],
                   [np.sin(alpha),np.cos(alpha),0],
                   [0,0,1]])
    Ry = np.array([[np.cos(beta),0,np.sin(beta)],
                   [0,1,0],
                   [-1*np.sin(beta),0,np.cos(beta)]])
    Rx = np.array([[1,0,0],
                   [0,np.cos(gamma),-1*np.sin(gamma)],
                   [0,np.sin(gamma),np.cos(gamma)]])
    R = np.dot(np.dot(Rz,Ry),Rx)

    x_1 =  x_1.T.reshape(3,1)
    x_2 = x_2.T.reshape(3,1)
    
    return (x_2 - np.dot(R,x_1)).T.reshape(3)
sol = fsolve(solve,[0,0,0])
a1,b1,c1 = sol[0],sol[1],sol[2]
a1d,b1d,c1d = convert_deg(a1,b1,c1)
print(a1d,b1d,c1d)


-41.062913795413124 5765.546112808945 35.455164399949105


# Analytical solution <br>
Our initial vector is [0,root(3),0] which will lie on y axis on root(3) <br>
We want to land at vector [1,1,1] by ZYX rotations. <br>
We move as follows, <br>
First rotate -45 deg around z axis, <br>
Then rotate around x-axis untill we land at [1,1,1] <br>
The angle of ratation will be <br>
arcsin(x_coord_destination/np.sqrt(3)) = arcsin(1/np.sqrt(3)) <br>
(From analysis on paper) <br>
Thus, alpha = -45 degree, beta = 0, and
gamma = arcsin(1/np.sqrt(3))

In [13]:
ansa = 7*np.pi/4 
ansb = 0
ansc = np.arcsin(1/np.sqrt(3))

Verifying if the answer is correct by intialising fsovle with answer +- 5

In [14]:
low = int(5*np.pi/180) #converting to radians
high = int(np.pi/180-5*np.pi/180)

In [15]:
for i in range(low,high+1):
    sol = fsolve(solve,[ansa+i,ansb+i,ansc+i])
    a2,b2,c2 = sol[0],sol[1],sol[2]
    a2d,b2d,c2d = convert_deg(a2,b2,c2)
    print("initialized with ans ",ansa+i,ansb+i,ansc+i,"\n angles got in degrees are",a2d,b2d,c2d )
    x_2_obtained_case2 = reconstructMat(a2,b2,c2) @ x_1.T 
    print("x_2_obtained_case2",x_2_obtained_case2)

initialized with ans  5.497787143782138 0 0.6154797086703875 
 angles got in degrees are 315.0 0.0 35.26438968275467
x_2_obtained_case2 [1. 1. 1.]


In [16]:
# From Section 2.2, use the function which takes Euler angles and gives Rotation matrix as output.
# Uncomment and replace `r_mat` with the name of the function. (Do NOT edit anything else)

x_2_obtained_case1 = reconstructMat(a1,b1,c1) @ x_1.T #TODO: replace r_mat
x_2_obtained_case2 = reconstructMat(a2,b2,c2) @ x_1.T #TODO: replace r_mat
test = True #TODO: Set this as True

In [17]:
# DON'T EDIT
if test == True:
    case1_test = np.isclose(x_2_obtained_case1,  np.array([1.0,1.0,1.0]))
    case2_test = np.isclose(x_2_obtained_case2,  np.array([1.0,1.0,1.0]))
    print("For case 1, it is " + str(bool(case1_test[0] and case1_test[1] and case1_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")
    print("For case 2, it is " + str(bool(case2_test[0] and case2_test[1] and case2_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")

For case 1, it is True that I could end up at (1,1,1) after rotation.
For case 2, it is True that I could end up at (1,1,1) after rotation.
